In [8]:
import PySimpleGUI as sg
import os.path
import coin as c
import numpy as np
import payment_menagment as pm
from decimal import *


Management Payment imported succesfully


In [9]:
class Ticket:
    def __init__(self, time, price):
        self._time = time
        self._price = Decimal(str(price))
        
    def get_price(self):
        return self._price

In [10]:
class Ticket_Machine(Payment_Menagement):
    def __init__(self):
        super().__init__()
    def calculate_credit(self, ticket):
        self.credit += ticket.get_price()
        
    def get_credit(self):
        return self.credit
    
    def calculate_payment(self, moneta):
        self.payment += moneta.get_value()
        if self.payment >= self.credit:
            self.change(self.payment-self.credit)
    
        
    def get_payment(self):
        return self.payment
    
    def add_ticket(self, ticket):
        tm.calculate_credit(ticket)
    
    def get_amount_to_pay(self):
        return tm.get_credit()-tm.get_payment()
    
        
    
    

In [12]:
tm = Ticket_Machine()
mode =0

layout_tickets = [[sg.Text("Wybierz bilety:")],
          [sg.Text("Normalny")],
          [sg.Button("20min - 3zł"),sg.Button("40min - 4zł"),sg.Button("60min -5zł")],
          [sg.Text("Ulgowy")],
          [sg.Button("20min - 2zł"),sg.Button("40min - 3zł"),sg.Button("60min -4zł")],
          [sg.Text('Do zapłaty: '),sg.Text('00', key='-creditInfoT-')],
          [sg.Button("Cofnij"),sg.Button("Dalej")]]

layout_pay = [[sg.Text("Wrzuć pieniądze: ")],
          [sg.Text('Do zapłaty: '), sg.Text("00",key='-creditInfoP-')],  
          [sg.Button("1zł"),sg.Button("2zł"),sg.Button("5zł")],
          [sg.Button("10zł"), sg.Button("20zł"), sg.Button("50zł")],
          [sg.Button("1gr"),sg.Button("2gr"),sg.Button("5gr")],
          [sg.Button("10gr"),sg.Button("20gr"),sg.Button("50gr")],
          [sg.Button("End")] ]

text_input = "AHAAA"   
my_new_theme= {'BACKGROUND': '#ffe6ff',
               'TEXT': 'black',
               'INPUT': '#DDE0DE',
               'SCROLL': '#E3E3E3',
               'TEXT_INPUT': 'black',
               'BUTTON': ('#000000', '#ff80d5'),
               'PROGRESS': ('#01826B', '#D0D0D0'),
               'BORDER': 1,
               'SLIDER_DEPTH': 0,
               'PROGRESS_DEPTH': 0}


sg.theme_add_new('MyNewTheme', my_new_theme)
sg.theme('My New Theme')

layouts = [[sg.Column(layout_tickets, key='-tickets-'), sg.Column(layout_pay, visible=False, key='-pay-')]]

# Create the window
window = sg.Window("Ticket Machine", layouts)

def change_layout(current_layout, next_layout):
    window[current_layout].update(visible=False)
    window[next_layout].update(visible=True)
    

def manage_tickets_layout():
    if event == "20min - 3zł" or event == "40min - 3zł":
        tm.add_ticket(Ticket(20,3))
    if event == "40min - 4zł" or event == "60min -4zł":
        tm.add_ticket(Ticket(40,4))
    if event == "20min - 2zł":
        tm.add_ticket(Ticket(20,2))
    if event == "60min -5zł":
        tm.add_ticket(Ticket(60,5))
    if event == "Dalej":
        if tm.get_credit() == 0:
            sg.popup("Wybierz bilet")
        else:    
            global mode
            mode=1
            change_layout(f'-tickets-',f'-pay-')
    window['-creditInfoT-'].update(value = str(tm.get_credit()))
            
    
def manage_payment_layout():
    window[f'-creditInfoP-'].update(value = str(tm.get_amount_to_pay()))
    if event == "1zł":
        tm.calculate_payment(c.Coin(1, "PLN"))
    if event == "2zł":
        tm.calculate_payment(c.Coin(2, "PLN"))
    if event == "5zł":
        tm.calculate_payment(c.Coin(5, "PLN"))
    if event == "10zł":
        tm.calculate_payment(c.Coin(10, "PLN"))
    if event == "20zł":
        tm.calculate_payment(c.Coin(20, "PLN"))
    if event == "50zł":
        tm.calculate_payment(c.Coin(50, "PLN"))
    if event == "0.1zł":
        tm.calculate_payment(c.Coin(0.1, "PLN"))
    if event == "0.2zł":
        tm.calculate_payment(c.Coin(0.2, "PLN"))
    if event == "0.5zł":
        tm.calculate_payment(c.Coin(0.5, "PLN"))
    window[f'-creditInfoP-'].update(value = str(tm.get_amount_to_pay()))
    

# Create an event loop
while True:
    event, values = window.read()
    if event == "Cofnij" or event == sg.WIN_CLOSED or event == "End":
        break
    if mode == 0:
        manage_tickets_layout()
    if mode == 1:
        manage_payment_layout()
    
    

    
    

window.close()